# Task 4 Random Circuits


Design a function that generates a random quantum circuit by considering as parameters the number of qubits, the number of depths, and the base of gates to be used.
You could only use the quantum gates of 1 and 2 qubits

### Installing necessary packages

Before we begin, you will need to install some prerequisites into your environment. Run the cell below to complete these installations. At the end, the cell outputs will be cleared.

In [ ]:
!pip install -U -r resources/requirements.txt

from IPython.display import clear_output
clear_output()

#### Importing necessary librarys

In [ ]:
import numpy as np

from qiskit.circuit import QuantumRegister, ClassicalRegister, QuantumCircuit
from qiskit.circuit import Reset
from qiskit.circuit.library.standard_gates import (IGate, U1Gate, U2Gate, U3Gate, XGate,
                                                   YGate, ZGate, HGate, SGate, SdgGate, TGate,
                                                   TdgGate, RXGate, RYGate, RZGate, CXGate,
                                                   CYGate, CZGate, CHGate, CRZGate, CU1Gate,
                                                   CU3Gate, SwapGate, RZZGate,
                                                   CCXGate, CSwapGate)
from qiskit.circuit.exceptions import CircuitError
from qiskit.util import deprecate_arguments

In [ ]:
def gates_division(gates):
    
    # function to seperate given gates into 1,2 qubits gates and 1,2,3 parameter gates 
    one_q_ops = [IGate, U1Gate, U2Gate, U3Gate, XGate, YGate, ZGate,
                 HGate, SGate, SdgGate, TGate, TdgGate, RXGate, RYGate, RZGate]
    one_param = [U1Gate, RXGate, RYGate, RZGate, RZZGate, CU1Gate, CRZGate]
    two_param = [U2Gate]
    three_param = [U3Gate, CU3Gate]
    two_q_ops = [CXGate, CYGate, CZGate, CHGate, CRZGate,
                 CU1Gate, CU3Gate, SwapGate, RZZGate]
    one_q_opsg,one_paramg,two_paramg,three_paramg,two_q_opsg=[],[],[],[],[]
    for gate in gates:
        if gate in one_q_ops:
            one_q_opsg.append(gate)
        if gate in one_param:
            one_paramg.append(gate)
        if gate in two_param:
            two_paramg.append(gate)
        if gate in three_param:
            three_paramg.append(gate)
        if gate in two_q_ops:
            two_q_opsg.append(gate)
      
    return one_q_opsg,one_paramg,two_paramg,three_paramg,two_q_opsg


In [ ]:
def random_circuit(num_qubits, depth,basis_gates, measure=False,
                   reset=False):
    """Generate random circuit of arbitrary size and form.

    This function will generate a random circuit by randomly selecting gates
    from the set of standard gates in :mod:`qiskit.extensions`. For example:

    .. jupyter-execute::

        circ = random_circuit(2, 2, measure=True)
        circ.draw(output='mpl')

    Args:
        num_qubits (int): number of quantum wires
        depth (int): layers of operations (i.e. critical path length)
        basis_gates(list): A list that contains the basis gates to generate the quantum circuit
        measure (bool): if True, measure all qubits at the end
        reset (bool): if True, insert middle resets

    Returns:
        QuantumCircuit: constructed circuit"""

    #calling the gates divison function 
    one_q_ops,one_param,two_param,three_param,two_q_ops=gates_division(basis_gates)
    
    # creating a quantum circuit
    qr = QuantumRegister(num_qubits, 'q')
    qc = QuantumCircuit(num_qubits)

    if measure :
        cr = ClassicalRegister(num_qubits, 'c')
        qc.add_register(cr)

    if reset:
        one_q_ops += [Reset]


    seed = np.random.randint(0, np.iinfo(np.int32).max)
    rng = np.random.default_rng(seed)

    # apply arbitrary random operations at every depth
    for _ in range(depth):
        # choose either 1 or 2 qubits for the operation
        remaining_qubits = list(range(num_qubits))
        while remaining_qubits:
            max_possible_operands = min(len(remaining_qubits),2)
            num_operands = rng.choice(range(max_possible_operands)) + 1
            rng.shuffle(remaining_qubits)
            operands = remaining_qubits[:num_operands]
            remaining_qubits = [q for q in remaining_qubits if q not in operands]
            if num_operands == 1:
                operation = rng.choice(one_q_ops)
            elif num_operands == 2:
                operation = rng.choice(two_q_ops)
            if operation in one_param:
                num_angles = 1
            elif operation in two_param:
                num_angles = 2
            elif operation in three_param:
                num_angles = 3
            else:
                num_angles = 0
            angles = [rng.uniform(0, 2 * np.pi) for x in range(num_angles)]
            register_operands = [qr[i] for i in operands]
            op = operation(*angles)


            qc.append(op, register_operands)

    if measure:
        qc.measure(qr, cr)

    return qc

In [ ]:
num_qubits =5
depth=3
basis_gates=[IGate, U1Gate, U2Gate, XGate, ZGate,HGate, SGate, SdgGate, TdgGate, RYGate, RZGate,CYGate,U3Gate]

circ=random_circuit(num_qubits, depth,basis_gates)
circ.draw('mpl')


In [ ]:
circ.depth()